### Preprocesamiento

In [76]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import json


conversión de valor de texto de los test a enteros, asignar 'Zalamea De La Serena' al codigo postal 6430 y descartar filas sin codigo postal ni geo location

In [77]:
# Definir el diccionario de mapeo
value_test_mapping = {
    'Dudoso': -2,
    'Error': -1,
    'Invalido': -1,
    'Inválido': -1,
    'Negativo': 0,
    'Positivo': 1,
    'Positivo Fuerte': 2
}

# Leer el archivo CSV
file_path = "data/new_data.csv" 
data = pd.read_csv(file_path)

# Mapear y convertir valores de 'Value test'
data['Value test'] = data['Value test'].map(value_test_mapping)
data['Value test'] = data['Value test'].fillna(0).astype(int)

# Eliminar filas donde 'Postal code' y 'Geo Location' estén vacíos
data_cleaned = data.dropna(subset=['Postal code', 'Geo Location'], how='all')

# Modificar 'City' si el código postal es 6430
data_cleaned['City'] = data_cleaned.apply(
    lambda row: 'Zalamea De La Serena' if row['Postal code'] == 6430 else row['City'], axis=1
)

# Agrupar por 'Postal code' y sumar los valores de 'Value test'
data_cleaned = data_cleaned.groupby('Postal code', as_index=False).agg({
    'Value test': 'sum',  # Sumar los valores de Value test
    'City': 'first',  # Mantener el primer valor de City por grupo
    'Geo Location': 'first',  # Mantener el primer valor de Geo Location por grupo
    # Puedes agregar otras columnas aquí, especificando cómo se deben agregar
})



Hace consultas a la api de "open cage data" (https://opencagedata.com/) para aquellas columnas que tienen geo location peor no tienen codigo postal, obtienen este último a través de la localización.

In [78]:
# Inicializa el geocoder con la clave de API
OCG = OpenCageGeocode('e5133987de494dd393d5de1370554b2f')

# Función para obtener el código postal a partir de Geo Location
def get_postal_code(geo_location):
    try:
        # Convertir la cadena JSON en un diccionario
        location = json.loads(geo_location)
        lat = location['latitude']
        lng = location['longitude']
        # Realizar la búsqueda inversa
        results = OCG.reverse_geocode(lat, lng)
        if results and 'postcode' in results[0]['components']:
            return results[0]['components']['postcode']
    except Exception as e:
        print(f"Error procesando {geo_location}: {e}")
    return None

# Agregar códigos postales donde faltan pero existe Geo Location
data_cleaned['Postal code'] = data_cleaned.apply(
    lambda row: get_postal_code(row['Geo Location']) if pd.isna(row['Postal code']) and pd.notna(row['Geo Location']) else row['Postal code'],
    axis=1
)



Hace consultas a la api de "open cage data" (https://opencagedata.com/) para aquellas columnas que tienen geo location peor no tienen geolocalizacion y la obtiene a traves de su codigo postal.

Agrupa en el csv archivo agrupado todos los codigos postales y suma el valor de los resultados de los tests.

In [79]:
def get_geo_location(postal_code):
    try:
        # Convertir el código postal a cadena
        postal_code_str = str(int(postal_code))  # Eliminar decimales innecesarios
        results = OCG.geocode(postal_code_str)
        if results:
            lat = results[0]['geometry']['lat']
            lng = results[0]['geometry']['lng']
            return f"{lat},{lng}"
    except Exception as e:
        print(f"Error procesando {postal_code}: {e}")
    return None

# Rellenar la columna 'Geo Location' donde falte pero exista 'Postal code'
data_cleaned['Geo Location'] = data_cleaned.apply(
    lambda row: get_geo_location(row['Postal code']) if pd.isna(row['Geo Location']) and pd.notna(row['Postal code']) else row['Geo Location'],
    axis=1
)

# Agrupar por 'Postal code' y sumar los valores de 'Value test'
grouped_data = data_cleaned.groupby('Postal code', as_index=False).agg({
    'Value test': 'sum',  # Sumar los valores de Value test
    'City': 'first',  # Mantener el primer valor de City por grupo
    'Geo Location': 'first',  # Mantener el primer valor de Geo Location por grupo
    # Puedes agregar otras columnas aquí, especificando cómo se deben agregar
})

# Guardar el archivo agrupado con las sumas
grouped_output_path = "data/archivo_agrupado.csv"
grouped_data.to_csv(grouped_output_path, index=False)

# Guardar el archivo completo procesado
processed_output_path = "data/archivo_completo.csv"
data_cleaned.to_csv(processed_output_path, index=False)


Crea un nuevo csv agrupando por codigo postal y guardando el numero de apariciones de cada tipo de resultado por codigo postal.

In [80]:
# Leer el archivo CSV
file_path = "data/new_data.csv" 
data = pd.read_csv(file_path)

# Crear un contador de cada tipo de resultado por código postal
result_counts = data.pivot_table(
    index='Postal code',  # Agrupar por código postal
    columns='Value test',  # Contar por cada tipo de resultado
    aggfunc='size',  # Contar las ocurrencias
    fill_value=0  # Rellenar los valores faltantes con 0
).reset_index()

# Renombrar las columnas para mayor claridad
result_counts.columns.name = None  # Quitar el nombre del índice de columnas
result_counts.rename(columns={
    -2: 'Dudoso',
    -1: 'Error',
    0: 'Negativo',
    1: 'Positivo',
    2: 'Positivo Fuerte'
}, inplace=True)

# Agregar columnas faltantes con 0 si no existen en los datos
for col in ['Dudoso', 'Error', 'Invalido', 'Inválido', 'Negativo', 'Positivo', 'Positivo Fuerte']:
    if col not in result_counts.columns:
        result_counts[col] = 0

# Reordenar las columnas
result_counts = result_counts[['Postal code', 'Dudoso', 'Error', 'Invalido', 'Inválido', 'Negativo', 'Positivo', 'Positivo Fuerte']]

# Guardar el resultado en un nuevo archivo CSV
output_path = "data/all_results_by_postal_code.csv"
result_counts.to_csv(output_path, index=False)